In [1]:
import streamlit as st
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import ast
from surprise.dump import load

# Cargar el modelo entrenado desde el archivo guardado
modelo_guardado = "models/modelo_user_item.pkl"
loaded_model = load(modelo_guardado)[1]

# Cargar los datos desde el archivo Parquet
file_path = 'data/data_user_item.parquet'
table = pq.read_table(file_path)
reviews = table.to_pandas()

In [2]:
def obtener_usuarios_con_items_no_calificados():
    user_ids = reviews['user_id'].unique()
    usuarios_con_items_no_calificados = []

    for user_id in user_ids:
        items_calificados = reviews[reviews['user_id'] == user_id]['business_id'].unique()
        items_no_calificados = reviews[~reviews['business_id'].isin(items_calificados)]['business_id'].unique()

        if len(items_no_calificados) > 0:
            usuarios_con_items_no_calificados.append((user_id, len(items_no_calificados)))

    # Crear un DataFrame con los usuarios que tienen ítems no calificados
    df_usuarios_con_items_no_calificados = pd.DataFrame(usuarios_con_items_no_calificados, columns=['user_id', 'num_items_no_calificados'])
    return df_usuarios_con_items_no_calificados

# Ejemplo de uso
df_usuarios_con_items_no_calificados = obtener_usuarios_con_items_no_calificados()
print("Usuarios con ítems no calificados:")
print(df_usuarios_con_items_no_calificados)


Usuarios con ítems no calificados:
            user_id  num_items_no_calificados
0      1.155553e+20                      2203
1      1.168684e+20                      2205
2      1.089802e+20                      2205
3      1.047022e+20                      2205
4      1.017705e+20                      2205
...             ...                       ...
78572  1.174809e+20                      2205
78573  1.144085e+20                      2205
78574  1.048164e+20                      2205
78575  1.005802e+20                      2205
78576  1.177883e+20                      2205

[78577 rows x 2 columns]


In [5]:
df_usuarios_con_items_no_calificados.to_parquet('data/df_usuarios_con_items_no_calificados.parquet', index=False)